In [13]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import cv2

import numpy as np

In [ ]:

img_height, img_width = 64, 64
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)

train_path = '../Datasets/Eyes_Data/TrainingSet'
test_path = '../Datasets/Eyes_Data/TestSet'
cv_path = '../Datasets/Eyes_Data/CVSet'


train_datagen  = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom transformation
    horizontal_flip=True  # Random horizontal flip
)


cv_datagen  = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom transformation
    horizontal_flip=True  # Random horizontal flip
)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size
)

val_gen = cv_datagen.flow_from_directory(
    cv_path,  # Using the new validation set
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size
)

test_gen = datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size,
    shuffle=False  # So you can match predictions to filenames
)


Found 1704 images belonging to 2 classes.
Found 4103 images belonging to 2 classes.
Found 4232 images belonging to 2 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 1)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])



model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 682,753 (2.60 MB)

 Trainable params: 682,753 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [6]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,  # <- CV set used here
    callbacks=[],  # (Optional) Add EarlyStopping or ReduceLROnPlateau if needed
)


Epoch 1/10


c:\Users\Anas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/54 ━━━━━━━━━━━━━━━━━━━━ 14s 241ms/step - accuracy: 0.5192 - loss: 0.6920 - val_accuracy: 0.5211 - val_loss: 0.6832
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 13s 239ms/step - accuracy: 0.6244 - loss: 0.6688 - val_accuracy: 0.7602 - val_loss: 0.6227
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 231ms/step - accuracy: 0.7648 - loss: 0.5901 - val_accuracy: 0.8391 - val_loss: 0.4448
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 231ms/step - accuracy: 0.8256 - loss: 0.4370 - val_accuracy: 0.8706 - val_loss: 0.3268
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 224ms/step - accuracy: 0.8456 - loss: 0.3752 - val_accuracy: 0.8745 - val_loss: 0.3009
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 225ms/step - accuracy: 0.8525 - loss: 0.3303 - val_accuracy: 0.8813 - val_loss: 0.2974
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 13s 235ms/step - accuracy: 0.8764 - loss: 0.3087 - val_accuracy: 0.8813 - val_loss: 0.3011
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 13s 237ms/step - accuracy: 0.8755 - loss: 0.3024 - val_accuracy: 0.870

In [7]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy:.2f}")


133/133 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.9026 - loss: 0.2734
Test Accuracy: 0.91


In [31]:
img_path = ["cc.jpg" , "oo.jpg" , "oo2.jpg"]  # Replace with the image you want to classify


for path in img_path:
    print(f"Processing: {path}")
    
    # Load image in grayscale
    img = load_img(path, color_mode="grayscale", target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    
    # Reshape to (1, 64, 64, 1)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    
    if prediction < 0.5:
        print(f"Prediction: {prediction[0][0]:.4f} — 👁️ Closed Eyes")
    else:
        print(f"Prediction: {prediction[0][0]:.4f} — 👁️ Open Eyes")

Processing: cc.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction: 0.4396 — 👁️ Closed Eyes
Processing: oo.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction: 0.8988 — 👁️ Open Eyes
Processing: oo2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction: 0.0214 — 👁️ Closed Eyes


In [12]:
model.save("open_close_eyes_model.h5")

In [34]:


# === Load your trained model ===
#model = load_model("open_close_eyes_model.h5")  # Replace with actual path
img_size = (64, 64)  # Model input size

# === Load Haar cascades ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# === Read image ===
img = cv2.imread("o1.jpg")  # Replace with your image path
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# === Detect faces ===
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

for (x, y, w, h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    # === Detect eyes in face ROI ===
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        # Eye region must be in upper half of face
        if ey > h // 2:
            continue

        # Optional: filter eyes based on size
        if ew < 10 or eh < 10 or ew > w // 2:
            continue
        eye_gray = roi_gray[ey:ey+eh, ex:ex+ew]
        eye_resized = cv2.resize(eye_gray, img_size)  # Resize to 64x64
        eye_array = eye_resized.astype("float32") / 255.0  # Normalize
        eye_array = np.expand_dims(eye_array, axis=-1)  # Add channel dim
        eye_array = np.expand_dims(eye_array, axis=0)   # Add batch dim

        # === Predict ===
        prediction = model.predict(eye_array)[0][0]
        label = "Open" if prediction >= 0.5 else "Closed"

        # === Draw results ===
        cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
        cv2.putText(roi_color, label, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2)

# === Display final result ===
cv2.imshow("Eye Detection & Classification", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
